In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from statistics import mean
import pandas as pd

import torch as th

from tabpfn import TabPFNClassifier

from tab_pfn.networks import SCM, TabPFN
from tab_pfn.metrics import AccuracyMeter

import warnings

warnings.filterwarnings("ignore")

In [ ]:
classifier = TabPFNClassifier(device='cuda', N_ensemble_configurations=32)

my_tab_pfn = TabPFN(100, 10, 256, 512, 1024, 4, 12)
my_tab_pfn.load_state_dict(th.load("/home/samuel/PycharmProjects/TabPFN/out_2/model_48127.pt"))
my_tab_pfn.cuda()
my_tab_pfn.eval()

n_datasets = 128

accuracy_meter_original = AccuracyMeter(None)
accuracy_meter_our = AccuracyMeter(None)
nb = 0

for _ in tqdm(range(n_datasets)):
    scm = SCM(100, (2, 10), False)
    x, y = scm(1024)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
    
    try:
        classifier.fit(x_train, y_train)
        p_eval = th.tensor(classifier.predict_proba(x_test))[None]
        
        accuracy_meter_original.add(p_eval.cpu(), y_test[None])
        
        x_train = x_train[None].cuda()
        y_train = y_train[None].cuda()
        x_test = x_test[None].cuda()
        y_test = y_test[None]
        
        p_eval = my_tab_pfn(x_train, y_train, x_test)
        
        accuracy_meter_our.add(p_eval.cpu(), y_test.cpu())
    except ValueError:
        pass

In [ ]:
print("original", accuracy_meter_original.accuracy())
print("our", accuracy_meter_our.accuracy())

In [ ]:
classifier = TabPFNClassifier(device='cuda', N_ensemble_configurations=32)

df = pd.read_csv("/home/samuel/Téléchargements/mfeat-karhunen_zip/data/mfeat-karhunen_csv.csv", sep=",")

x = df.iloc[:, :-1]
y = df.iloc[:, -1]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42)

classifier.fit(x_train, y_train - 1)
p_eval = classifier.predict_proba(x_test)

accuracy_meter = AccuracyMeter(None)
accuracy_meter.add(th.tensor(p_eval), th.tensor(y_test.tolist()) - 1)

In [ ]:
accuracy_meter.accuracy()

In [ ]:
p_eval.shape

In [ ]:
th.tensor(y_test.tolist())